## Data Cleaning

In [1]:
import pandas as pd
import datetime as dt

USvideos_data = pd.read_csv('Resources/USvideos.zip')
CAvideos_data = pd.read_csv('Resources/CAvideos.zip')

# Add country column
USvideos_data['country'] = 'USA'
CAvideos_data['country'] = 'Canada'

# Convert 'trending_date' column datatype from object to datetime
USvideos_data['trending_date'] = '20'+ USvideos_data['trending_date']
USvideos_data['trending_date'] = USvideos_data['trending_date'].str.replace('.','-')
USvideos_data['trending_date'] = USvideos_data['trending_date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%d-%m'))

CAvideos_data['trending_date'] = '20'+ CAvideos_data['trending_date']
CAvideos_data['trending_date'] = CAvideos_data['trending_date'].str.replace('.','-')
CAvideos_data['trending_date'] = CAvideos_data['trending_date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%d-%m'))

# Set Data Range: 2018 June trending data is the latest data available at this moment
date_rangeUS = (USvideos_data['trending_date'] > '2018-06-01') & (USvideos_data['trending_date'] <= '2018-06-30')
USvideos_data = USvideos_data.loc[date_rangeUS]

date_rangeCA = (CAvideos_data['trending_date'] > '2018-06-01') & (CAvideos_data['trending_date'] <= '2018-06-30')
CAvideos_data = CAvideos_data.loc[date_rangeCA]

# Find & Count the null columns
null_columns = USvideos_data.columns[USvideos_data.isnull().any()]
print(USvideos_data[null_columns].isnull().sum())

# Locate and inspect null values within data
print(USvideos_data[USvideos_data.isnull().any(axis=1)][null_columns])

description    20
dtype: int64
      description
38484         NaN
38526         NaN
38689         NaN
38729         NaN
38892         NaN
38932         NaN
39096         NaN
39136         NaN
39302         NaN
39339         NaN
39506         NaN
39543         NaN
39709         NaN
39744         NaN
39916         NaN
40125         NaN
40325         NaN
40538         NaN
40739         NaN
40945         NaN


In [2]:
# Complete the null data value
datatype_USVideos = USvideos_data.dtypes.to_dict()
USvideos_data['description'].fillna('None', inplace=True, downcast=datatype_USVideos)

datatype_CAVideos = USvideos_data.dtypes.to_dict()
CAvideos_data['description'].fillna('None', inplace=True, downcast=datatype_CAVideos)

In [3]:
# Export and backup cleaned data
export_USvideos = USvideos_data.to_csv('Output/USvideos_clean.csv')
export_CAvideos = CAvideos_data.to_csv('Output/CAvideos_clean.csv')

In [4]:
# Combine two dataset
combine_df = pd.merge(USvideos_data,CAvideos_data, how='outer')

# Load file with category name data
category_idname = pd.read_csv('Resources/YT_Category_idname.csv', header=None)
category_idname.columns = ['category_id','category_name']

# Combine data
combine_df = combine_df.merge(category_idname, how='left', on='category_id')
# Convert publish_time as date
combine_df['publish_time'] = pd.to_datetime(combine_df['publish_time'])
combine_df['publish_time'] = combine_df['publish_time'].dt.date

# Drop unnecessary columns
combine_df.drop(columns=['comments_disabled','ratings_disabled','video_error_or_removed'], inplace=True)

In [5]:
# Export and backup cleaned data
export_combined = combine_df.to_csv('Output/Combined_data.csv')

## Export Data for postgreSQL database

In [6]:
# Inspect duplicates for ERD ideas
find_dupe = combine_df[combine_df.duplicated(['video_id'],keep=False)]
print(find_dupe.video_id.count())
find_dupe.sort_values(['video_id']).head(4)

4117


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,description,country,category_name
1026,-0CMnp02rNY,2018-06-07,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,2018-06-04,"ellen|""ellen degeneres""|""the ellen show""|""elle...",605506,7848,232,354,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,Ocean's 8 star Mindy Kaling dished on bringing...,USA,Entertainment
1658,-0CMnp02rNY,2018-06-10,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,2018-06-04,"ellen|""ellen degeneres""|""the ellen show""|""elle...",792613,9720,330,413,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,Ocean's 8 star Mindy Kaling dished on bringing...,USA,Entertainment
820,-0CMnp02rNY,2018-06-06,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,2018-06-04,"ellen|""ellen degeneres""|""the ellen show""|""elle...",475965,6531,172,271,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,Ocean's 8 star Mindy Kaling dished on bringing...,USA,Entertainment
1859,-0CMnp02rNY,2018-06-11,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,2018-06-04,"ellen|""ellen degeneres""|""the ellen show""|""elle...",800359,9773,332,423,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,Ocean's 8 star Mindy Kaling dished on bringing...,USA,Entertainment


In [7]:
find_dupe.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'description', 'country', 'category_name'],
      dtype='object')

In [8]:
video_basics = combine_df[['trending_date','title','channel_title','category_id','views','likes','publish_time']]

video_stats = combine_df[['video_id','tags','dislikes','comment_count','thumbnail_link','description','country']]
video_stats['country'] = video_stats['country'].replace({'USA':0,'Canada':1})
video_stats = video_stats[['video_id','dislikes','comment_count','tags','thumbnail_link','description','country']]

country_id = pd.DataFrame({'country_id':[0,1],'country_name':['USA','Canada']})

C:\Users\hjyoo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [9]:
trend_id = combine_df['trending_date'].value_counts().to_frame().reset_index().drop(columns='trending_date')\
            .rename(columns={'index':'trending_date'}).sort_values('trending_date').reset_index(drop=True)
trend_id = trend_id.reset_index()
trend_id.rename(columns={'index':'trend_id'},inplace=True)

In [10]:
publish_id = combine_df['publish_time'].value_counts().to_frame().reset_index().drop(columns='publish_time')\
                .rename(columns={'index':'publish_time'}).sort_values('publish_time').reset_index(drop=True)
publish_id.reset_index(inplace=True)
publish_id.rename(columns={'index':'publish_id'},inplace=True)

In [11]:
channel_id = combine_df['channel_title'].value_counts().to_frame().reset_index().drop(columns='channel_title')\
                .rename(columns={'index':'channel_title'}).sort_values('channel_title').reset_index(drop=True)
channel_id.reset_index(inplace=True)
channel_id.rename(columns={'index':'channel_id'},inplace=True)

In [12]:
video_basics = trend_id.merge(video_basics,how='left').drop(columns='trending_date')
video_basics = video_basics.merge(publish_id,how='left').drop(columns='publish_time')
video_basics = channel_id.merge(video_basics,how='left').drop(columns='channel_title')

In [13]:
video_basics = video_basics[['trend_id','title','channel_id','category_id','publish_id','views','likes']]

In [14]:
video_basics = video_basics.sort_values('title').reset_index(drop=True)

In [15]:
exp_sql_v_basics = video_basics.to_csv('SQL_Resources/Video_basic.csv',index=False)
exp_sql_v_stats = video_stats.to_csv('SQL_Resources/Video_stat.csv',index=False)
exp_sql_ctgry_id = category_idname.to_csv('SQL_Resources/Category.csv',index=False)
exp_sql_ctry_id = country_id.to_csv('SQL_Resources/Country.csv',index=False)
exp_sql_trnd_id = trend_id.to_csv('SQL_Resources/Trend.csv',index=False)
exp_sql_pub_id = publish_id.to_csv('SQL_Resources/Publish.csv',index=False)
exp_sql_title_id = channel_id.to_csv('SQL_Resources/Channel.csv',index=False)